clone and cd into repo, nshepperd's fork https://github.com/nshepperd/gpt-2

In [1]:
!pip install tensorflow-gpu==1.13.1

In [2]:
import tensorflow as tf
tf.__version__

ImportError: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/usr/lib/python3.6/imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "/usr/lib/python3.6/imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: libcuda.so.1: cannot open shared object file: No such file or directory


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/errors

for some common reaso

In [3]:
!git clone https://github.com/ncoop57/gpt-2.git ./nct_data/gpt-2

Cloning into './nct_data/gpt-2'...
remote: Enumerating objects: 301, done.
remote: Total 301 (delta 0), reused 0 (delta 0), pack-reused 301
Receiving objects: 100% (301/301), 4.40 MiB | 13.16 MiB/s, done.
Resolving deltas: 100% (165/165), done.


In [4]:
cd nct_data/gpt-2

/tf/prototypes/gpt-2/nct_data/gpt-2


Install requirements

In [5]:
!pip install -r requirements.txt

     |████████████████████████████████| 604kB 4.5MB/s eta 0:00:01
     |████████████████████████████████| 61kB 41.5MB/s eta 0:00:01
     |████████████████████████████████| 51kB 28.1MB/s eta 0:00:01
     |████████████████████████████████| 143kB 19.6MB/s eta 0:00:01
     |████████████████████████████████| 163kB 14.0MB/s eta 0:00:01
     |████████████████████████████████| 122kB 616kB/s eta 0:00:01
  Stored in directory: /root/.cache/pip/wheels/2a/1a/4d/6b30377c3051e76559d1185c1dbbfff15aed31f87acdd14c22
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex


Download the model data

In [6]:
!python download_model.py 345M

Fetching checkpoint: 1.00kit [00:00, 442kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 8.93Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 370kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [01:41, 14.0Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 4.32Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 8.84Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 6.43Mit/s]                                                       


encoding

In [7]:
!export PYTHONIOENCODING=UTF-8

In [8]:
!apt install wget -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
wget is already the newest version (1.19.4-1ubuntu2.2).
0 upgraded, 0 newly installed, 0 to remove and 9 not upgraded.



Let's get our train on! In this case the file is A Tale of Two Cities (Charles Dickens) from Project Gutenberg. To change the dataset GPT-2 models will fine-tune on, change this URL to another .txt file, and change corresponding part of the next cell. Note that you can use small datasets if you want but you will have to be sure not to run the fine-tuning for too long or you will overfit badly. Roughly, expect interesting results within minutes to hours in the 1-10s of megabyte ballpark, and below this you may want to stop the run early as fine-tuning can be very fast.

In [9]:
!wget https://raw.githubusercontent.com/micheletufano/NeuralCodeTranslator/master/dataset/bug-fixes/medium/train/fixed.txt

--2019-06-12 19:29:58--  https://raw.githubusercontent.com/micheletufano/NeuralCodeTranslator/master/dataset/bug-fixes/medium/train/fixed.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.20.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.20.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15798070 (15M) [text/plain]
Saving to: ‘fixed.txt’

fixed.txt           100%[===================>]  15.07M  13.5MB/s    in 1.1s    

2019-06-12 19:29:59 (13.5 MB/s) - ‘fixed.txt’ saved [15798070/15798070]

--2019-06-12 19:29:59--  http://methods.txt/
Resolving methods.txt (methods.txt)... failed: Name or service not known.
wget: unable to resolve host address ‘methods.txt’
FINISHED --2019-06-12 19:29:59--
Total wall clock time: 1.6s
Downloaded: 1 files, 15M in 1.1s (13.5 MB/s)


In [10]:
!ls

CONTRIBUTORS.md  LICENSE	    fixed.txt	      train-horovod.py
DEVELOPERS.md	 README.md	    models	      train.py
Dockerfile.cpu	 download_model.py  requirements.txt
Dockerfile.gpu	 encode.py	    src


In [16]:
!tf_upgrade_v2 --infile ./src/memory_saving_gradients.py --outfile ./src/memory_saving_gradients.py

INFO line 13:8: Renamed 'tf.GraphKeys' to 'tf.compat.v1.GraphKeys'
INFO line 89:26: Renamed 'tf.get_collection' to 'tf.compat.v1.get_collection'
INFO line 336:7: `name` passed to `name_scope`. Because you may be re-entering an existing scope, it is not safe to convert automatically,  the v2 name_scope does not support re-entering scopes by name.

INFO line 336:7: Renamed 'tf.name_scope' to 'tf.compat.v1.name_scope'
INFO line 339:6: Renamed 'tf.get_default_graph' to 'tf.compat.v1.get_default_graph'
TensorFlow 2.0 Upgrade Script
-----------------------------
Converted 1 files
Detected 0 issues that require attention
--------------------------------------------------------------------------------


Make sure to read the detailed log 'report.txt'



In [11]:
!pip install toposort


Start training, add --model_name '345M' to use 345 model

In [17]:
!PYTHONPATH=src ./train.py --dataset ./fixed.txt --model_name '345M'

Traceback (most recent call last):
  File "./train.py", line 17, in <module>
    import memory_saving_gradients
  File "/tf/prototypes/gpt-2/nct_data/gpt-2/src/memory_saving_gradients.py", line 5, in <module>
    import tensorflow.contrib.graph_editor as ge
ModuleNotFoundError: No module named 'tensorflow.contrib'


Save our checkpoints to start training again later

In [0]:
!cp -r /content/gpt-2/checkpoint/ /content/drive/My\ Drive/

cp: cannot create directory '/content/drive/My Drive/': No such file or directory


Load your trained model for use in sampling below (117M or 345M)

In [0]:
!cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/117M/

In [0]:
!cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/345M/

Generate conditional samples from the model given a prompt you provide -  change top-k hyperparameter if desired (default is 40),  if you're using 345M, add "--model-name 345M"

In [0]:
!python3 src/interactive_conditional_samples.py --top_k 40 --model_name "345M"

2019-05-29 01:17:42.237182: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-05-29 01:17:42.237511: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x33f0680 executing computations on platform Host. Devices:
2019-05-29 01:17:42.237554: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-05-29 01:17:42.432852: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-05-29 01:17:42.433377: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x33effa0 executing computations on platform CUDA. Devices:
2019-05-29 01:17:42.433421: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2019-05-29 01:17:42.433828: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433] Found de

To check flag descriptions, use:

In [0]:
!python3 src/interactive_conditional_samples.py -- --help

Generate unconditional samples from the model,  if you're using 345M, add "--model-name 345M"

In [0]:
!python3 src/generate_unconditional_samples.py --model_name "345M" | tee /tmp/samples

To check flag descriptions, use:

In [0]:
!python3 src/generate_unconditional_samples.py -- --help